In [1]:
# Opencv Library is very useful to capture and work with images and videos
import cv2
#Mediapipe is framework for building machine learning pipelines for processing time-series data like video
import mediapipe as mp
#import time
#Math is used to calculate distance
import math
#numpy can manage multidimensional array
import numpy as np
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
#We are directly importing Audio controlling library as they are pre build
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
cap = cv2.VideoCapture(0)
#We are now starting to capture videos

In [2]:
pTime = 0
cTime = 0
#drawing utility
mpHands =  mp.solutions.hands
hands = mpHands.Hands()
#Used to detect hands
mpDraws = mp.solutions.drawing_utils
#mpDraw is to mark coordinates
#Audio Utility
devices = AudioUtilities.GetSpeakers()
#Configuring the speakers with AudioUtilities
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
volRange = volume.GetVolumeRange()
minVol = volRange[0]
maxVol = volRange[1]
#Seting up what is maximum and volume of this program will be like

In [3]:
while True:
    #This loop will work all the time unless we force stop it
    success , img = cap.read()
    imgRGB = cv2.cvtColor(img , cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            lmList =[]
            for id , lm in enumerate(handLms.landmark):
                #we made List with name lmList to storing all the marks 
                #print(id , lm)
                #We can uncomment the print function above to print the coordinated with their respective ID
                h , w , c = img.shape
                cx , cy = int(lm.x*w) ,int(lm.y*h)
                #print(id,cx,cy)
                #We can uncomment the print above to see x and y co-ordinated of all the landmarks
                lmList.append([id, cx, cy])
                #we are now storing the coordinated in list we formed earlier
                #if id == 1:
                #    cv2.circle(img ,(cx,cy) ,20, (255 , 0,125) ,cv2.FILLED )
                mpDraws.draw_landmarks(img , handLms , mpHands.HAND_CONNECTIONS)
                #print(results.multi_hand_landmarks)

            if lmList:    
                x1 , y1 = lmList[4][1] , lmList[4][2]
                x2 , y2 = lmList[8][1] , lmList[8][2]
                x3 , y3 = lmList[12][1] , lmList[12][2]
                #We are now extracting 8th and 4th point which represent forefinger's and thumbs's apex
                cv2.circle(img , (x1 ,y1) ,10 , (0,0,255) ,cv2.FILLED)
                cv2.circle(img , (x2 ,y2) ,10 , (0,0,255) ,cv2.FILLED)
                cv2.circle(img , (x3 ,y3) ,10 , (0,0,255) ,cv2.FILLED)
                #openCV reads in BGR so the point 4th and 8th are completely red
                cv2.line(img, (x1,y1) , (x2,y2) , (0 , 0, 255) ,3)
                #forming a line from point 4 to 8th which is also red
                cv2.line(img, (x1,y1) , (x3,y3) , (255 , 0, 255) ,3)
                z1 = (x1 + x2)//2
                z2 = (y1 + y2)//2
                z3 = (x1 + x3)//2
                z4 = (y1 + y3)//2
                cv2.circle(img , (z1,z2),15 ,(0,255,0) , cv2.FILLED )
                #this point is middle groung of coordinate 4th and 8th which is marked as green
                length = math.hypot(x2- x1 , y2- y1)
                test = math.hypot(x3- x1 , y3- y1)
                #print(length)

                if length < 100 or length > 250:
                    cv2.circle(img , (z1 , z2) ,15 ,(255,255,255) , cv2.FILLED)
                #Change middle colour to white if distance is in non operable ranga
            if test < 100:
                #Now volume will only get modified if Index finger and thumb are close to each other giving use more freedom
                cv2.circle(img , (z3 , z4) ,15 ,(0,255,255) , cv2.FILLED)
                #When the condition is met we will have a yellow circle appear
                vol = np.interp(length , [50 ,250] ,[minVol ,maxVol])
                volBar = np.interp(length , [50,250] ,[400 ,150])
                volPer = np.interp(length , [50 , 250] , [0,100])
            #print(vol)
                print(int(length) , vol)
            #print(volume.GetVolumeRange())
                volume.SetMasterVolumeLevel(vol, None)
                cv2.rectangle(img ,(50,150) , (85,400) ,(0,255,0) , 3)
                cv2.rectangle(img ,(50,int(volBar)) , (85,400) ,(255,0,0) , cv2.FILLED)
                cv2.putText(img , f'{int(volPer)}%', (40,450) ,cv2.FONT_HERSHEY_PLAIN,3 ,(255,255,255) ,3)
            #Making a rectangle as volume bar of blue colour with white text
            
            #if lmList:
            #    print(lmList[4][1] , lmList[8])

                
            
            
#    cTime = time.time()
#    fps = 1/(cTime - pTime)
#    pTime = cTime
#    cv2.putText(img , str(int(fps)) , (10,70) ,cv2.FONT_HERSHEY_PLAIN , 3, (255,0,255) , 3)

    

    cv2.imshow("Image" , img)
    cv2.waitKey(1)

119 -62.41400170180255
119 -62.68671358227658
119 -62.86185162258056
118 -63.20157748669423
120 -62.24022160707332
122 -61.34564977770191
121 -61.68444461380823
123 -60.78270522896204
48 -96.0
41 -96.0
59 -91.6434134635355
62 -90.17812883134259
39 -96.0
32 -96.0
32 -96.0
37 -96.0
44 -96.0
43 -96.0
57 -92.30520626543682
78 -82.43405797800354
86 -78.25103594099609
105 -69.5794327679666
124 -60.431632555524914
122 -61.390840306313905
152 -46.731179892126015
182 -32.38191967407639
213 -17.354045379274694
220 -14.397818204357165
247 -1.3001634373492692
257 0.0
261 0.0
240 -4.382755611457327
266 0.0
260 0.0
243 -3.193205677066885
280 0.0
270 0.0
255 0.0
263 0.0
262 0.0
247 -1.3001634373492692
250 0.0
258 0.0
258 0.0
259 0.0
257 0.0
261 0.0
265 0.0
267 0.0
252 0.0
250 0.0
255 0.0
262 0.0
258 0.0
260 0.0
299 0.0
299 0.0
218 -15.12466829611455
219 -14.525529155155283
217 -15.482273273860372
213 -17.441622477732224
211 -18.71545033915588
211 -18.430366742810378
210 -18.81559013365745
208 -19.871

268 0.0
271 0.0
273 0.0
275 0.0
311 0.0
102 -70.84978128227709
130 -57.48011516325386
129 -57.944334666365876
135 -54.951587259949214
127 -58.605016491573195
125 -59.856172386519994
100 -71.61276201310928
79 -81.87397739076367
96 -73.8975054904835
92 -75.74618660499414
90 -76.55005638668791
84 -79.31592940719918
73 -84.48973106269118
77 -82.94350259401193
82 -80.22656162713614
65 -88.35638453020894
73 -84.95671248298756
75 -83.86903820820707
81 -80.83364709345534
89 -77.25573722708509
86 -78.32836936235876
72 -85.23726132768019
78 -82.18952526084868
83 -80.05603925497623
92 -75.52546796198975
91 -76.21463258119215
107 -68.30241785150876
87 -77.97326565149274
96 -73.51676431228135
100 -71.5770632860831
92 -75.63958521384183
89 -76.88541778005961
106 -68.65794706091312
116 -64.13410342615094
113 -65.47191549302323
103 -70.40412920413637
107 -68.17555789012293
102 -70.95301844149836
94 -74.82641479802604
94 -74.5949430129198
57 -92.22213831123784
80 -81.34584110344657
90 -76.6296691273861

KeyboardInterrupt: 